In [1]:
from kafka import KafkaProducer
from kafka.errors import KafkaError
import pandas as pd
from binance.client import Client
import datetime
import requests
from minio import Minio
import tempfile
from s3fs.core import S3FileSystem
from binance_config import *

In [2]:
# Binance client
api_key = API_KEY
api_secret = SECRET_KEY
client = Client(api_key=api_key, api_secret=api_secret)

In [2]:
# Minio client
mclient = Minio('10.152.183.240:9000', access_key='access-key', secret_key='secret-key', secure=False)

In [10]:
# Kafka client
producer = KafkaProducer(bootstrap_servers=['kafka.kafka.svc.cluster.local:9092'])

In [20]:
# s3fs client
s3 = S3FileSystem(
    anon=False,
    key='access-key',
    secret='secret-key',
    use_ssl=False,
    client_kwargs={'endpoint_url': 'http://10.152.183.240:9000'})

In [21]:
bnbbtc = client.get_symbol_info('BNBBTC')

In [21]:
payload = client.get_symbol_ticker(symbol='BNBBTC')

In [ ]:
producer.send('test', key=bytearray(payload['symbol'], 'utf-8'), value=bytearray(payload['price'], 'utf-8'))

In [3]:
mclient.fput_object('repo', 'wbs-kafka-test.py', 'untitled1.py')

('451b8aedae38a37370a8d3bff7e2ce52', None)

In [18]:
with s3.open('data/test3.csv', 'w') as f:
    df.to_csv(f)

In [17]:
tickers = client.get_all_tickers()

df = pd.DataFrame(tickers)

In [ ]:
# Asynchronous by default
future = producer.send('test', b'raw_bytes')

# Block for 'synchronous' sends
try:
    record_metadata = future.get(timeout=10)
except KafkaError:
    # Decide what to do if produce request failed...
    log.exception()
    pass

# Successful result returns assigned partition and offset
print (record_metadata.topic)
print (record_metadata.partition)
print (record_metadata.offset)

# produce keyed messages to enable hashed partitioning
producer.send('test', key=b'foo', value=b'bar')

# encode objects via msgpack
producer = KafkaProducer(value_serializer=msgpack.dumps)
producer.send('test', {'key': 'value'})

In [32]:
producer.send('test', key=b'foo', value=b'bar')

In [ ]:
# produce json messages
producer = KafkaProducer(value_serializer=lambda payload: json.dumps(payload).encode('ascii'))
producer.send('test', {'key': 'value'})

In [ ]:
# produce asynchronously
for _ in range(100):
    producer.send('my-topic', b'msg')

def on_send_success(record_metadata):
    print(record_metadata.topic)
    print(record_metadata.partition)
    print(record_metadata.offset)

def on_send_error(excp):
    log.error('I am an errback', exc_info=excp)
    # handle exception

# produce asynchronously with callbacks
producer.send('my-topic', b'raw_bytes').add_callback(on_send_success).add_errback(on_send_error)

# block until all async messages are sent
producer.flush()

# configure multiple retries
producer = KafkaProducer(retries=5)

In [ ]:
start_date = datetime.datetime.strptime('1 Jan 2016', '%d %b %Y')
today = datetime.datetime.today()

def binanceBarExtractor(symbol):
    print('working...')
    filename = '{}_MinuteBars.csv'.format(symbol)

    klines = bclient.get_historical_klines(symbol, Client.KLINE_INTERVAL_1MINUTE, start_date.strftime("%d %b %Y %H:%M:%S"), today.strftime("%d %b %Y %H:%M:%S"), 1000)
    data = pd.DataFrame(klines, columns = ['open_timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_timestamp', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    # data['open_time'] = pd.to_datetime(data['open_time'], unit='ms')
    # data['close_time'] = pd.to_datetime(data['close_time'], unit='ms')
    # data = data.reset_index()

    # data.set_index('open_time', inplace=True)
    data.to_csv(filename)
    print('finished!')

In [1]:
from kafka import KafkaProducer
from binance import ThreadedWebsocketManager
import json

# Binance client
api_key = "wtqoMQp21dTGKL4qrZeHaxv0UZETf55sHOqaWLgu6wAKfmB3N7hTBPjGoMFpHAv3"
api_secret = "v1YKCDgI11Tl7czwUUgAfBvDWaZjg12hiN3HTMVen8EqDdZuVJxPiPZ3scKW1pBa"
twm = ThreadedWebsocketManager(api_key=api_key, api_secret=api_secret)

# Kafka Client
producer = KafkaProducer(bootstrap_servers=['kafka.kafka.svc.cluster.local:9092'], value_serializer=lambda m: json.dumps(m).encode('ascii'))

symbol = 'BTCUSD'

def handle_socket_message(msg):

    producer.send('test', {symbol : msg})
    print(msg.json())

twm.start_kline_socket(callback=handle_socket_message, symbol=symbol)

twm.join()

KeyboardInterrupt: 